### <최적 정책을 추정하기 위한 비활성 정책 MC 제어> 

모든 $s \in S, a \in A(s)$에 대해 초기화 : 
- $Q(s,a) \in R$ (임의의 값으로 설정) 
- $C(s,a)$ <- 0 
- $\pi(s)$ <- $argmax_aQ(s,a)$ (최대가 되는 a가 여러 개인 경우 한 가지 행동만 선택) 

(각 에피소드에 대해) 무한 루프 : 
- b <- 임의의 소프트 정책 
- 정책 b를 활용하여 에피소드를 생성 : $S_0, A_0, R_1, ..., S_{T-1}, A_{T-1}, R_T$ 
- G <- 0 
- W <- 1 
- 에피소드의 각 단계에 대한 루프, t = T-1, T-2, ..., 0 : 

> G <- $\gamma G + R_{t+1}$ 

> $C(S_t, A_t) <- C(S_t, A_t) + W$

> $Q(S_t, A_t)$ <- $Q(S_t, A_t) + \frac {W}{C(S_t, A_t)} (G-Q(S_t, A_t))$

> $\pi(S_t)$ <- $argmax_a Q(S_t,a)$

> $A_t \neq \pi(S_t)$ 이면 루프를 종료 

> W <- $W \frac{1}{b(A_t|S_t)}$



**구현해야 하는 것** 
- Q(s,a) : 기존 형식 유지 
- C(s,a) : 기존 형식 유지

- b <- 정책 $\pi$ 가 보증된 임의의 정책 : 모든 확률을 동일하게 설ㅈ어  

- $\pi(s,a)$ : 내부 함수로 구현할 것. dic 형태로 구현 


**필요한 것** 
- 외부함수 reward func

- def choose_random_max(lst) : 최대값이 2개 이상인 경우, 임의로 1개의 최대값을 만들어낸 행동 a를 산출 
 

**함수/데이터의 형태** 
- class epsilon_soft_MC : 

> def __init__(self, S, A, reward_func,  epsilon=0.001, gamma = 0.9, num_episode = 10, len_episode = 20) 

> def choice_sample(self, list, prob) : 기존 함수 유지 

> def make_episode(self, start_s, T, pi) : 기존 함수 유지  

> def update_returns(self) : 비활성 MC에 따라서 구현 

**전제**
- 상태 s에서 행동 a를 했을 때 결정론적으로 s' 상태로 변화한다. 

<외부 함수> 
- def reward_func(s',a,s) : 



In [1]:
# 테스트 용 임시 데이터 
S = list(range(100)) 
A = list(range(-5,5))

In [2]:
from collections import defaultdict
import random
import numpy as np

In [3]:
# 외부용 함수 reward_func 간략 구현 (이전 예시 활용)
def reward_func(next_s, a, s) : 
    # next_s 와 s의 차이가 짝수이면 +1, 홀수면 -1 
    # 단, a의 크기에 반비례함. 
    if abs(next_s - s) %2 == 0 : reward = 1 
    else : reward = -1
    
    if a == 0 : 
        return 0 
    else : 
        return reward / a # 즉, a가 양수이며 짝수이며, 가능한 작을 때 (=2) 일 때 최대의 보상이 주어지도록 설정 
    
#최대값이 2개 이상인 경우, 임의로 1개의 최대값을 만들어낸 행동 a를 산출 
def choose_random_max(lst) :
    max_arg = np.where(np.array(lst) >= max(lst))
    return random.choice(list(max_arg)[0]) #max_arg가 array 형태로 안에 있는 list를 꺼내기 위해 [0] 사용 
    

In [10]:
# 활성 접촉 MC 제어 코드 참고 

class epsilon_soft_MC : 
    def __init__(self, S, A, reward_func, epsilon = 0.001, gamma = 0.9, num_episode = 10, len_episode = 20) : 
        self.S = S 
        self.A = A 
        self.epsilon = epsilon 
        self.reward_func = reward_func
        self.pi = [0]*len(self.S)   # 값을 수정할 수 있는 데이터 형식으로 바꿔야 함. 결정론 방식이라 list로 구현 
        self.b, self.C = self.initiate_b() 
        self.gamma = 0.9 
        self.num_episode = num_episode
        self.T = len_episode
        
        self.set_episode = [] # 추후에 어떤 episode들이 있었나 확인용 
        
        self.Q, self.return_lst, self.s_prob = self.initiate() 
    
    
    def initiate_b(self) : 
        # pi를 보증하는 b 정책 초기화하기. 예측이기 때문에 b 정책이 바뀔 일은 없음.
        # C(s,a) 도 같이 초기화 
        b_dic = defaultdict(float)
        c_dic = defaultdict(float)
        for s in self.S :
            for a in self.A :  
                b_dic[(s,a)] = 1/len(self.A)
                c_dic[(s,a)] = 0
        return b_dic, c_dic
    
    def initiate(self) : # V 대신 Q(a,s)를 초기화해야 함. 
        Q = defaultdict(float)
        s_prob = defaultdict(list)
        for s in self.S : 
            s_prob[s] = [0]*len(self.A)
            for index, a in enumerate(self.A) : 
                Q[(a,s)] = 0 
                s_prob[s][index] = self.b[(s,a)] # pi에 대한 식 수정 필요

        return_lst = [[0]*len(self.A)]*len(self.S) # return_lst의 데이터 형식 변경
        return Q, return_lst, s_prob
    
    def choice_sample(self, s) : 
        a= random.choices(self.A, weights = self.s_prob[s])
        
        # 수정. a[0] +s 가 s의 범위 안에 들어오도록 수정 
        return a[0], min(max(s+a[0],0), 99)  
    
    def make_episode(self, start_s, T) :
        s = start_s
        episode = {"S" : [], "A" : [], "R" : []}
        episode["R"].append(0) # R_0 값 부여 
        for _ in range(T) : 
            episode["S"].append(s)
            a, next_s = self.choice_sample(s)
            r = self.reward_func(next_s, a, s)
            episode["A"].append(a)
            episode["R"].append(r)
            s = next_s
        return episode 
 
    def update_returns(self) : # epsilon - soft 행동 가치 함수에 맞춰 변경
        for _ in range(self.num_episode) :
            # 시작 s는 시작 가정에 따라 랜덤하게 산출하겠음. 
            start_s = random.choice(self.S)
            episode = self.make_episode(start_s, self.T)
            G = 0 
            W = 1 
            
            # G와 returns(S_t)를 업데이트 하는 순서가 각각 역순이라 따로 구현
            for t in reversed(range(self.T)) : 
                s, a = episode["S"][t], episode["A"][t]
                G = self.gamma *G + episode["R"][t]
                self.C[(s,a)] = self.C[(s,a)] + W 
                self.Q[(s,a)] = self.Q[(s,a)] + W/self.C[(s,a)] *(G - self.Q[(s,a)])
                q_a_list = [] 
                for _ in self.A : q_a_list.append(self.Q[(s,_)])
                self.pi[s] = choose_random_max(q_a_list)
                if a != self.pi[s] : break
                W = W/self.b[(s,a)]

                

        
                
        

In [16]:
# 점검 
test = epsilon_soft_MC(S,A,reward_func, num_episode = 100000, len_episode = 1000) 
test.update_returns() 
#print(test.Q.values())
print(test.pi)
#print(test.return_lst)


[9, 9, 9, 8, 9, 9, 7, 4, 4, 4, 7, 4, 4, 2, 4, 4, 4, 2, 0, 4, 7, 2, 7, 7, 4, 0, 4, 7, 2, 4, 4, 7, 4, 4, 4, 8, 7, 9, 3, 5, 5, 1, 5, 1, 9, 1, 2, 4, 0, 8, 5, 4, 9, 6, 3, 7, 6, 0, 3, 0, 1, 1, 8, 6, 3, 7, 1, 5, 2, 0, 5, 8, 1, 4, 0, 0, 0, 3, 0, 8, 0, 0, 0, 0, 5, 1, 0, 0, 6, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0]
